In [1]:

# %pip install pygis
import ee
import geemap
geemap.ee_initialize()

## Earth Engine data types
### Image
##### Loading Earth Engine images

In [2]:
image = ee.Image('USGS/SRTMGL1_003')
image

In [3]:
image.getInfo()

{'type': 'Image',
 'bands': [{'id': 'elevation',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [1296001, 417601],
   'crs': 'EPSG:4326',
   'crs_transform': [0.0002777777777777778,
    0,
    -180.0001388888889,
    0,
    -0.0002777777777777778,
    60.00013888888889]}],
 'version': 1641990767055141,
 'id': 'USGS/SRTMGL1_003',
 'properties': {'system:visualization_0_min': '0.0',
  'type_name': 'Image',
  'keywords': ['dem',
   'elevation',
   'geophysical',
   'nasa',
   'srtm',
   'topography',
   'usgs'],
  'thumb': 'https://mw1.google.com/ges/dd/images/SRTM90_V4_thumb.png',
  'description': '<p>The Shuttle Radar Topography Mission (SRTM, see <a href="https://onlinelibrary.wiley.com/doi/10.1029/2005RG000183/full">Farr\net al. 2007</a>)\ndigital elevation data is an international research effort that\nobtained digital elevation models on a near-global scale. This\nSRTM V3 product (SRTM Plus) is provided by NASA JP

##### Visualizing Earth Engine images

In [11]:
Map = geemap.Map(center = [52.52, 13.40], zoom = 8)
image = ee.Image('USGS/SRTMGL1_003')
vis_params = {
    'min': 0,
    'max': 6000,
    'palette': ['006633', 'C4A484', '5C4033', 'A67B5B', 'D2B48C']}
Map.addLayer(image, vis_params, 'SRTM DEM')
Map

Map(center=[52.52, 13.4], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

##### Loading Cloud GeoTIFFs

In [25]:
Map = geemap.Map(height = "300px")

B3 = "gs://gcp-public-data-landsat/LC08/01/044/034/LC08_L1TP_044034_20131228_20170307_01_T1/LC08_L1TP_044034_20131228_20170307_01_T1_B3.TIF"
B4 = "gs://gcp-public-data-landsat/LC08/01/044/034/LC08_L1TP_044034_20131228_20170307_01_T1/LC08_L1TP_044034_20131228_20170307_01_T1_B4.TIF"
B5 = "gs://gcp-public-data-landsat/LC08/01/044/034/LC08_L1TP_044034_20131228_20170307_01_T1/LC08_L1TP_044034_20131228_20170307_01_T1_B5.TIF"

URLs = [B3, B4, B5]

collection = geemap.load_GeoTIFFs(URLs)

image = collection.toBands().rename(["Green", "Red", "NIR"]).selfMask()

vis = {
    "bands": ["NIR", "Red", "Green"], 
    "min": 100, 
    "max": 12000, 
    "gamma": 0.8
    }

Map.addLayer(image, vis, "Image")
Map.centerObject(image.geometry(), 8)

ndvi = image.normalizedDifference(["NIR", "Red"])
ndvi_vis = {"min": 0, "max": 1, "palette": ["blue", "green"]}
Map.addLayer(ndvi, {}, "NDVI")

Map


Map(center=[37.4683111265957, -122.14225670841434], controls=(WidgetControl(options=['position', 'transparent_…

In [33]:

# Create Map
Map = geemap.Map(zoom=8)

# Landsat 8 Band URLs
B3 = "gs://gcp-public-data-landsat/LC08/01/052/013/LC08_L1GT_052013_20130713_20170503_01_T2/LC08_L1GT_052013_20130713_20170503_01_T2_B3.TIF"
B4 = "gs://gcp-public-data-landsat/LC08/01/052/013/LC08_L1GT_052013_20130713_20170503_01_T2/LC08_L1GT_052013_20130713_20170503_01_T2_B4.TIF"
B5 = "gs://gcp-public-data-landsat/LC08/01/052/013/LC08_L1GT_052013_20130713_20170503_01_T2/LC08_L1GT_052013_20130713_20170503_01_T2_B5.TIF"

URLs = [B3, B4, B5]

# Load the GeoTIFFs
collection = geemap.load_GeoTIFFs(URLs)

# Convert to multi-band image and rename
image = collection.toBands().rename(["Green", "Red", "NIR"]).unmask(0).unitScale(0, 10000)

# Auto-scale visualization
stats = image.reduceRegion(
    reducer=ee.Reducer.percentile([2, 98]), 
    geometry=image.geometry(),
    scale=100,  
    bestEffort=True 
)


min_val = stats.getNumber("NIR_p2").getInfo()
max_val = stats.getNumber("NIR_p98").getInfo()

vis = {
    "bands": ["NIR", "Red", "Green"], 
    "min": min_val, 
    "max": max_val
}

# Display the image
Map.addLayer(image, vis, "Landsat 8 Image")
Map.centerObject(image.geometry(), 8)

# Compute NDVI
ndvi = image.normalizedDifference(["NIR", "Red"])

# NDVI Visualization
ndvi_vis = {"min": 0, "max": 1, "palette": ["blue", "green"]}
Map.addLayer(ndvi, ndvi_vis, "NDVI")

Map


Map(center=[66.93533914960447, -118.99743489843317], controls=(WidgetControl(options=['position', 'transparent…

In [36]:

import os
os.environ["TITILER_ENDPOINT"] = "https://titiler.xyz"
Map = geemap.Map()

url = "https://github.com/opengeos/data/releases/download/raster/Libya-2023-07-01.tif"
geemap.cog_bounds(url)
geemap.cog_center(url)
geemap.cog_bands(url)
geemap.cog_tile(url)
Map.add_cog_layer(url, name="Fire (pre-event)")
url2 = "https://github.com/opengeos/data/releases/download/raster/Libya-2023-09-13.tif"
Map.add_cog_layer(url2, name="Fire (post-event)")

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [37]:
collection = ee.ImageCollection('COPERNICUS/S2_SR')
collection.limit(5)

In [40]:
Map = geemap.Map()
collection = ee.ImageCollection('COPERNICUS/S2_SR')
image = collection.median()

vis = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
}

Map.setCenter(52.52, 13.40, 12)
Map.addLayer(image, vis, 'Sentinel-2')
Map

Map(center=[13.4, 52.52], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [41]:
Map = geemap.Map()
collection = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate('2021-01-01', '2022-01-01')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))
)
image = collection.median()

vis = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
}

Map.setCenter(52.52, 13.40, 12)
Map.addLayer(image, vis, 'Sentinel-2')
Map

Map(center=[13.4, 52.52], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [42]:
Map = geemap.Map()

point = ee.Geometry.Point([1.5, 1.5])

lineString = ee.Geometry.LineString([[-35, -10], [35, -10], [35, 10], [-35, 10]])

linearRing = ee.Geometry.LinearRing(
    [[-35, -10], [35, -10], [35, 10], [-35, 10], [-35, -10]]
)

rectangle = ee.Geometry.Rectangle([-40, -20, 40, 20])

polygon = ee.Geometry.Polygon([[[-5, 40], [65, 40], [65, 60], [-5, 60], [-5, 60]]])

Map.addLayer(point, {}, 'Point')
Map.addLayer(lineString, {}, 'LineString')
Map.addLayer(linearRing, {}, 'LinearRing')
Map.addLayer(rectangle, {}, 'Rectangle')
Map.addLayer(polygon, {}, 'Polygon')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [43]:
Map = geemap.Map()

point = ee.Geometry.Point([1.5, 1.5])

lineString = ee.Geometry.LineString(
    [[-35, -10], [35, -10], [35, 10], [-35, 10]], None, False
)

linearRing = ee.Geometry.LinearRing(
    [[-35, -10], [35, -10], [35, 10], [-35, 10], [-35, -10]], None, False
)

rectangle = ee.Geometry.Rectangle([-40, -20, 40, 20], None, False)

polygon = ee.Geometry.Polygon(
    [[[-5, 40], [65, 40], [65, 60], [-5, 60], [-5, 60]]], None, False
)

Map.addLayer(point, {}, 'Point')
Map.addLayer(lineString, {}, 'LineString')
Map.addLayer(linearRing, {}, 'LinearRing')
Map.addLayer(rectangle, {}, 'Rectangle')
Map.addLayer(polygon, {}, 'Polygon')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [45]:
if Map.user_roi is not None:
    print(Map.user_roi.getInfo())

if Map.user_roi is not None:
    print(Map.user_roi)

In [46]:
polygon = ee.Geometry.Polygon(
    [[[-35, -10], [35, -10], [35, 10], [-35, 10], [-35, -10]]], None, False
)
polyFeature = ee.Feature(polygon, {'foo': 42, 'bar': 'tart'})

polyFeature

In [47]:
Map = geemap.Map()
Map.addLayer(polyFeature, {}, 'feature')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [48]:
feature = (
    ee.Feature(ee.Geometry.Point([-122.22599, 37.17605]))
    .set('genus', 'Sequoia')
    .set('species', 'sempervirens')
)
newDict = {'genus': 'Brachyramphus', 'presence': 1, 'species': 'marmoratus'}
feature = feature.set(newDict)
feature


In [49]:
prop = feature.get('species')
prop

In [53]:
Map = geemap.Map()
fc = ee.FeatureCollection('TIGER/2016/Roads')
Map.setCenter(-73.9596, 40.7688, 12)
Map.addLayer(fc, {}, 'Census roads')
Map

Map(center=[40.7688, -73.9596], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

In [54]:
fc.limit(3)

In [55]:
features = [
    ee.Feature(ee.Geometry.Rectangle(30.01, 59.80, 30.59, 60.15), {'name': 'Voronoi'}),
    ee.Feature(ee.Geometry.Point(-73.96, 40.781), {'name': 'Thiessen'}),
    ee.Feature(ee.Geometry.Point(6.4806, 50.8012), {'name': 'Dirichlet'}),
]
fromList = ee.FeatureCollection(features)

In [56]:
Map = geemap.Map()
states = ee.FeatureCollection('TIGER/2018/States')
feat = states.filter(ee.Filter.eq('NAME', 'Texas'))
Map.addLayer(feat, {}, 'Texas')
Map.centerObject(feat)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [57]:
texas = feat.first()
texas.toDictionary().getInfo()

{'ALAND': 676653171537,
 'AWATER': 19006305260,
 'DIVISION': '7',
 'FUNCSTAT': 'A',
 'GEOID': '48',
 'INTPTLAT': '+31.4347032',
 'INTPTLON': '-099.2818238',
 'LSAD': '00',
 'MTFCC': 'G4000',
 'NAME': 'Texas',
 'REGION': '3',
 'STATEFP': '48',
 'STATENS': '01779801',
 'STUSPS': 'TX'}

In [58]:
Map = geemap.Map()
states = ee.FeatureCollection('TIGER/2018/States')
fc = states.filter(ee.Filter.inList('NAME', ['California', 'Oregon', 'Washington']))
Map.addLayer(fc, {}, 'West Coast')
Map.centerObject(fc)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
region = Map.user_roi
if region is None:
    region = ee.Geometry.BBox(-88.40, 29.88, -77.90, 35.39)

fc = ee.FeatureCollection('TIGER/2018/States').filterBounds(region)
Map.addLayer(fc, {}, 'Southeastern U.S.')
Map.centerObject(fc)

# Map desplayed above

In [60]:
Map = geemap.Map(center=[40, -100], zoom=4)
states = ee.FeatureCollection("TIGER/2018/States")
Map.addLayer(states, {}, "US States")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [61]:
Map = geemap.Map(center=[40, -100], zoom=4)
states = ee.FeatureCollection("TIGER/2018/States")
image = ee.Image().paint(states, 0, 3)
Map.addLayer(image, {'palette': 'red'}, "US States")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [62]:
Map = geemap.Map(center=[40, -100], zoom=4)
states = ee.FeatureCollection("TIGER/2018/States")
style = {'color': '0000ffff', 'width': 2, 'lineType': 'solid', 'fillColor': 'FF000080'}
Map.addLayer(states.style(**style), {}, "US States")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [63]:
Map = geemap.Map(center=[40, -100], zoom=4)
states = ee.FeatureCollection("TIGER/2018/States")
vis_params = {
    'color': '000000',
    'colorOpacity': 1,
    'pointSize': 3,
    'pointShape': 'circle',
    'width': 2,
    'lineType': 'solid',
    'fillColorOpacity': 0.66,
}
palette = ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
Map.add_styled_vector(
    states, column="NAME", palette=palette, layer_name="Styled vector", **vis_params
)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [ ]:
Map = geemap.Map(center=[28.00142, -81.7424], zoom=13)
Map.add_basemap('HYBRID')

types = [
    "Freshwater Forested/Shrub Wetland",
    "Freshwater Emergent Wetland",
    "Freshwater Pond",
    "Estuarine and Marine Wetland",
    "Riverine",
    "Lake",
    "Estuarine and Marine Deepwater",
    "Other",
]

colors = [
    "#008837",
    "#7FC31C",
    "#688CC0",
    "#66C2A5",
    "#0190BF",
    "#13007C",
    "#007C88",
    "#B28653",
]

fillColor = [c + "A8" for c in colors]

fc = ee.FeatureCollection("projects/sat-io/open-datasets/NWI/wetlands/FL_Wetlands")
styled_fc = geemap.ee_vector_style(
    fc, column='WETLAND_TY', labels=types, fillColor=fillColor, color='00000000'
)

Map.addLayer(styled_fc, {}, 'NWI')
Map.add_legend(title='Wetland Type', labels=types, colors=colors)
Map


Map(center=[28.00142, -81.7424], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [68]:
Map.save('geemap.pdf')

In [70]:
fuels = [
    'Coal',
    'Oil',
    'Gas',
    'Hydro',
    'Nuclear',
    'Solar',
    'Waste',
    'Wind',
    'Geothermal',
    'Biomass',
]
colors = [
    '000000',
    '593704',
    'BC80BD',
    '0565A6',
    'E31A1C',
    'FF7F00',
    '6A3D9A',
    '5CA2D1',
    'FDBF6F',
    '229A00',
]

fc = ee.FeatureCollection("WRI/GPPD/power_plants").filter(
    ee.Filter.inList('fuel1', fuels)
)
styled_fc = geemap.ee_vector_style(fc, column="fuel1", labels=fuels, color=colors)

Map = geemap.Map(center=[40, -100], zoom=4)
Map.addLayer(styled_fc, {}, 'Power Plants')
Map.add_legend(title="Power Plant Fuel Type", labels=fuels, colors=colors)
Map



Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [72]:
types = ['I', 'U', 'S', 'M', 'C', 'O']
labels = ['Interstate', 'U.S.', 'State recognized', 'Common Name', 'County', 'Other']
colors = ['E31A1C', 'FF7F00', '6A3D9A', '000000', 'FDBF6F', '229A00']
width = [8, 5, 4, 2, 1, 1]

fc = ee.FeatureCollection('TIGER/2016/Roads')
styled_fc = geemap.ee_vector_style(
    fc, column='rttyp', labels=types, color=colors, width=width
)

Map = geemap.Map(center=[40.7424, -73.9724], zoom=13)
Map.addLayer(styled_fc, {}, 'Census Roads')
Map.add_legend(title='Route Type', labels=labels, colors=colors)
Map


Map(center=[40.7424, -73.9724], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

 ## Earth Engine Data Catalog
####  Searching for datasets

In [73]:
dataset_xyz = ee.Image('USGS/SRTMGL1_003')
Map.addLayer(dataset_xyz, {}, "USGS/SRTMGL1_003")

Map = geemap.Map()
dem = ee.Image('USGS/SRTMGL1_003')
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
Map.addLayer(dem, vis_params, 'SRTM DEM')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### Using the datasets module

In [75]:
from geemap.datasets import DATA 
Map = geemap.Map(center=[40, -100], zoom=4)
dataset = ee.Image(DATA.USGS_GAP_CONUS_2011)
Map.addLayer(dataset, {}, 'GAP CONUS')
Map



Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [76]:
from geemap.datasets import get_metadata

get_metadata(DATA.USGS_GAP_CONUS_2011)

HTML(value='\n            <html>\n            <body>\n                <h3>USGS GAP CONUS 2011</h3>\n          …

In [77]:
image = ee.Image('LANDSAT/LC09/C02/T1_L2/LC09_044034_20220503')
geemap.image_min_value(image)

In [78]:
geemap.image_stats(image)

In [79]:
Map = geemap.Map(center=(40, -100), zoom=4)
dem = ee.Image('USGS/SRTMGL1_003')
landsat7 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003').select(
    ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
)
states = ee.FeatureCollection("TIGER/2018/States")
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
Map.addLayer(dem, vis_params, 'SRTM DEM')
Map.addLayer(
    landsat7,
    {'bands': ['B4', 'B3', 'B2'], 'min': 20, 'max': 200, 'gamma': 2.0},
    'Landsat 7',
)
Map.addLayer(states, {}, "US States")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [81]:
url = "https://github.com/gee-community/geemap/blob/master/examples/data/countries.zip"
geemap.download_file(url)

Map = geemap.Map()
in_shp = "countries.shp"
Map.add_shp(in_shp, layer_name="Countries")
Map

Downloading...
From: https://raw.githubusercontent.com/gee-community/geemap/master/examples/data/countries.zip
To: d:\Personal_Projects\Geospatial\Other Tutorials\countries.zip
100%|██████████| 132k/132k [00:00<00:00, 735kB/s]


Extracting files...


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…